# QA1. Changing associations

In [1]:
from koselleck import *

In [2]:
dfnov=get_novelty_data()
words_signif = set(dfnov.query('p_peak<0.01').word)
len(words_signif),random.sample(words_signif,5)

(844, ['quest', 'bookseller', 'tome', 'fatigue', 'night'])

In [3]:

dfkw=get_keywords()
words_kw=set(dfkw.index)
len(words_kw),random.sample(words_kw,5)

(218, ['literature', 'economy', 'feudalism', 'native', 'commercialism'])

In [5]:
# words=words_kw | words_signif
words=get_valid_words()

In [6]:
# for x in ['reform','citizen','slavery','train','demand']:
#     print(x,x in words)

In [7]:
dfdec=get_pathdf_models().query('period_len==10')
# dfdec=get_pathdf_models().query('period_len==70')
dfdec

,corpus,period_start,period_end,path,path_vocab,run,period,period_len
1204,bpo,1680,1690,/home/ryan/github/koselleck/data/models/bpo/16...,/home/ryan/github/koselleck/data/models/bpo/16...,run_01,1680-1690,10
1211,bpo,1680,1690,/home/ryan/github/koselleck/data/models/bpo/16...,/home/ryan/github/koselleck/data/models/bpo/16...,run_02,1680-1690,10
1193,bpo,1680,1690,/home/ryan/github/koselleck/data/models/bpo/16...,/home/ryan/github/koselleck/data/models/bpo/16...,run_03,1680-1690,10
1202,bpo,1680,1690,/home/ryan/github/koselleck/data/models/bpo/16...,/home/ryan/github/koselleck/data/models/bpo/16...,run_04,1680-1690,10
1206,bpo,1680,1690,/home/ryan/github/koselleck/data/models/bpo/16...,/home/ryan/github/koselleck/data/models/bpo/16...,run_05,1680-1690,10
...,...,...,...,...,...,...,...,...
2191,bpo,1960,1970,/home/ryan/github/koselleck/data/models/bpo/19...,/home/ryan/github/koselleck/data/models/bpo/19...,run_21,1960-1970,10
2200,bpo,1960,1970,/home/ryan/github/koselleck/data/models/bpo/19...,/home/ryan/github/koselleck/data/models/bpo/19...,run_22,1960-1970,10
2198,bpo,1960,1970,/home/ryan/github/koselleck/data/models/bpo/19...,/home/ryan/github/koselleck/data/models/bpo/19...,run_23,1960-1970,10
2183,bpo,1960,1970,/home/ryan/github/koselleck/data/models/bpo/19...,/home/ryan/github/koselleck/data/models/bpo/19...,run_24,1960-1970,10


In [8]:
def do_return_neighborhoods(rowdf,words=None,topn=10):
    row=rowdf.iloc[0]
    m=load_model_row(row)
    if words is None: words=get_valid_words()
    words=set(words) & set(m.wv.key_to_index.keys())
    return pd.DataFrame([
        {
            'word':word,
            'neighborhood':', '.join(x for x,c in m.wv.most_similar(word,topn=topn))
        } for word in words
    ])

In [9]:
# do_return_neighborhoods(dfdec.iloc[:10], {'heart'}, 50)

In [10]:
def return_all_neighborhoods(dfdec, words=None, topn=50, num_proc=1):
    if words is None: words=get_valid_words()
    return pmap_groups(
        do_return_neighborhoods,
        dfdec.groupby(['period','run']),
        num_proc=num_proc,
        kwargs=dict(words=words,topn=topn)
    )

In [19]:
# get_keywords(just_words=True)
# get_distmatrix_rateofchange()

In [11]:
odf=return_all_neighborhoods(dfdec, words=words, num_proc=6)
# odf.to_pickle('data/data.all_neighborhoods.v6.pkl')

Mapping do_return_neighborhoods [x6]: 100%|██████████| 725/725 [1:34:08<00:00,  7.79s/it]  


In [13]:
df = odf
# df=pd.read_pickle('data/data.all_neighborhoods.v4.pkl').reset_index()
# df['period_int']=df.period.apply(lambda x:int(x[:4]))
# df['period2']=df.period_int.apply(prdz)
# df

In [14]:
def synth(df,max_rank=25):
    o=[]
    for (w,p),wdf in tqdm(df.groupby(['word','period'])):
        counts=Counter()
        ranks=defaultdict(list)
        for neighb in wdf.neighborhood:
            for i,w2 in enumerate(neighb.split(', ')):
#                 if max_rank and i>=max_rank: break
                ranks[w2]+=[i+1]
                counts[w2]+=1
        ow=[]
        for w2 in ranks:
            odx={
                'word':w,
                'period':p,
                'word2':w2,
                'rank':np.mean(ranks[w2]),
                'count':counts[w2]
            }
            ow.append(odx)
        owdf=pd.DataFrame(ow)
        owdf['rankcount']=owdf['count'] * (1/ (owdf['rank']+1))
        owdf['rank2']=(-owdf['rankcount']).rank()
        o.append(owdf)
    return pd.concat(o)
dfres=synth(df).set_index('word')
dfres

100%|██████████| 148115/148115 [30:43<00:00, 80.36it/s]   


,period,word2,rank,count,rankcount,rank2
word,,,,,,
abandonment,1780-1790,insuperable,17.25,4,0.219178,61.0
abandonment,1780-1790,exactions,2.00,1,0.333333,27.0
abandonment,1780-1790,friary,3.00,1,0.250000,49.0
abandonment,1780-1790,indemnification,34.00,4,0.114286,137.5
abandonment,1780-1790,uncandid,5.00,1,0.166667,100.5
...,...,...,...,...,...,...
zoology,1950-1960,morel,39.00,1,0.025000,403.5
zoology,1950-1960,limbless,41.00,1,0.023810,411.5
zoology,1950-1960,stabs,42.00,1,0.023256,418.5


In [15]:
dfres.to_pickle('data/data.neighborhoods.cache.v2.pkl')